# Using Llama 3.1 - Llama 3.1 405B Instruct

## Tech Stack
#### LangChain - An Open-source AI framework to help you build LLM powered applications
#### SingleStore Database - A robust all-in-one data platform for your AI/ML applications
#### Fireworks - Fireworks AI is a generative AI inference platform to run and customize models with industry-leading speed and production-readiness.

In [5]:
!pip install langchain_openai langchain_text_splitters singlestoredb langchain-community langchain_fireworks --quiet

In [7]:
import os
import dotenv
import requests
from bs4 import BeautifulSoup
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import CharacterTextSplitter
from langchain_community.vectorstores import SingleStoreDB

# Set API keys directly in the notebook
os.environ['OPENAI_API_KEY'] = 'Add Your Open AI API Key'

# Retrieve API keys from environment variables
openai_api = os.getenv("OPENAI_API_KEY")

dotenv.load_dotenv()
fireworks_api = os.getenv("Add Your Fireworks API Key")

# Extract text from Wikipedia article
url = "https://en.wikipedia.org/wiki/Manhattan_Project"
response = requests.get(url)
soup = BeautifulSoup(response.text, "html.parser")
text = soup.get_text(strip=True)

# Split text into chunks
text_splitter = CharacterTextSplitter(chunk_size=200, chunk_overlap=0, separator=" ")
text_chunks = text_splitter.split_text(text)
docs = text_splitter.create_documents(text_chunks)

## Create Embeddings of Our Data

In [29]:
embeddings = OpenAIEmbeddings(model="text-embedding-3-large", api_key=openai_api)
vectorstore = SingleStoreDB(
    OpenAIEmbeddings(),
    host="username:password@host_url:3306/sampledb"
)

## Store The Embeddings Within Our Database

In [30]:
# Setup connection url as environment variable
os.environ["SINGLESTOREDB_URL"] = "username:password@host_url:3306/sampledb"

# Load documents to the store
docsearch = SingleStoreDB.from_documents(
    docs,
    embeddings,
    table_name="llama3data",  # use table with a custom name
)

In [34]:
import os

# Set API keys directly in the notebook
os.environ['FIREWORKS_API_KEY'] = 'Add Your Fireworks API Key'

## Mention the Model

In [35]:
from langchain_fireworks import ChatFireworks

llm = ChatFireworks(
    model="accounts/fireworks/models/llama-v3p1-405b-instruct",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
    fireworks_api_key=fireworks_api,
)

## Create Prompts and Chain 

In [46]:
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough


def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


template = """
I am creating a YouTube video to demonstrate your proficiency in Retrieval-Augmented Generation (RAG). Please use the provided context to answer the questions accurately. Below is the context for you to reference:

{context}

Question: {question}

Answer Here:"""
prompt = PromptTemplate.from_template(template)

chain = (
    {
        "context": vectorstore.as_retriever() | format_docs,
        "question": RunnablePassthrough(),
    }
    | prompt
    | llm
    | StrOutputParser()
)

#### Ask the Query

In [47]:
chain.invoke("What was the role of the S-1 Committee in the Manhattan Project?")

"The S-1 Committee played a crucial role in the Manhattan Project as a secret research and development committee established in 1942 to investigate the possibility of developing an atomic bomb. The committee was responsible for coordinating and overseeing the early research efforts on nuclear fission, particularly the work of scientists such as Enrico Fermi, Ernest Lawrence, and Robert Oppenheimer.\n\nThe S-1 Committee was tasked with exploring the feasibility of using nuclear fission to create a bomb, and its members were responsible for identifying the key scientific and technical challenges that needed to be addressed. The committee's work laid the foundation for the eventual creation of the Manhattan Project, which was the large-scale research and development effort that produced the atomic bomb during World War II.\n\nThe S-1 Committee's specific responsibilities included:\n\n1. Coordinating research efforts: The committee brought together scientists from various universities and 

In [52]:
chain.invoke("Who were the leading scientists involved in the Manhattan Project?")

'Based on the context of the Manhattan Project, some of the leading scientists involved were:\n\n1. J. Robert Oppenheimer: Director of the Manhattan Project\'s Los Alamos Laboratory, often referred to as the "father of the atomic bomb."\n2. Enrico Fermi: Led the team that developed the first nuclear reactor, known as the Chicago Pile-1, and made significant contributions to the development of the atomic bomb.\n3. Ernest Lawrence: Invented the cyclotron, a type of particle accelerator, and was a key figure in the development of the electromagnetic separation method for uranium-235.\n4. Richard Feynman: A young physicist who worked in the theoretical division at Los Alamos and played a crucial role in the development of the atomic bomb.\n5. Klaus Fuchs: A German-born physicist who worked at Los Alamos and provided significant contributions to the development of the atomic bomb, although his involvement was later marred by controversy due to his espionage activities.\n6. Edward Teller: A 